In [ ]:
#DOWNLOADING THE DATA ON THE COLAB'S SERVER
import urllib.request
urllib.request.urlretrieve ("https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz","a.tar.gz")


('a.tar.gz', <http.client.HTTPMessage at 0x7b08e05b78b0>)

In [ ]:
import tarfile
tar = tarfile.open("a.tar.gz")
tar.extractall()
tar.close()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [ ]:
#forming a panda dataframe from the metadata file
data=pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")

In [ ]:
#head of the dataframe
data.head()

In [ ]:
#count of datapoints in each of the folders
data["fold"].value_counts()

# Examples of Data Preprocessing and the Features we will be creating and using


In [ ]:
from librosa import display
import librosa
#feature set
#This file is of a dog bark
y,sr=librosa.load("UrbanSound8K/audio/fold5/100032-3-0-0.wav")
mfccs = librosa.feature.mfcc(y, sr, n_mfcc=36)
melspectrogram =librosa.feature.melspectrogram(y=y, sr=sr, n_mels=36,fmax=8000)
chroma_stft=librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=36)
chroma_cq =librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=36)
chroma_cens =librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=36)
melspectrogram.shape,chroma_stft.shape,chroma_cq.shape,chroma_cens.shape,mfccs.shape
# TESTING THE DATA PREPROCESSING ON SAMPLE DATASETS TO UNDERSTAND THE PROCESS OF DATA COLLECTION FROM THE DATASET

In [ ]:
#MFCC of dog bark
import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
librosa.display.specshow(mfccs, x_axis='time')
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()

In [ ]:
#Melspectrogram of a dog bark
plt.figure(figsize=(10,4))
librosa.display.specshow(librosa.power_to_db(melspectrogram,ref=np.max),y_axis='mel', fmax=8000,x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel spectrogram')
plt.tight_layout()

In [ ]:
#Chromagram of dog bark
plt.figure(figsize=(10,4))
librosa.display.specshow(chroma_stft, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.title('Chromagram')
plt.tight_layout()

In [ ]:
Chroma cqt of a dog bark
plt.figure(figsize=(10,4))
librosa.display.specshow(chroma_cq, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.title('chroma_cqt')
plt.tight_layout()

In [ ]:
#Chroma cens of a dog bark
plt.figure(figsize=(10,4))
librosa.display.specshow(chroma_cens, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.title('chroma_cens')
plt.tight_layout()


# Data Preprocessing and the production of the features for the entire set

In [ ]:
#preprocessing using entire feature set
x_data=[ [] for _ in range(10) ]
y_label=[ [] for _ in range(10) ]
path="UrbanSound8K/audio/fold"
for i in tqdm(range(len(data))):
    fold_no=str(data.iloc[i]["fold"])
    file=data.iloc[i]["slice_file_name"]
    label=data.iloc[i]["classID"]
    filename=path+fold_no+"/"+file
    y,sr=librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=36).T,axis=0)
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=36,fmax=8000).T,axis=0)
    chroma_stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=36).T,axis=0)
    chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=36).T,axis=0)
    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=36).T,axis=0)
    features=np.reshape(np.vstack((mfccs,melspectrogram,chroma_stft,chroma_cq,chroma_cens)),(36,5))
    x_data[int(fold_no)-1].append(features)
    y_label[int(fold_no)-1].append(label)
    # USED OS TO HAVE THE COMPLETE ADDRESS OF THE FILENAME EACH TIME AND THEN LOADED THAT DATA INTO THE LIBROSA ONE BY ONE UrbanSound8K/audio/fold/fold_no/slice_file_name/class_ID
    # loaded these into the librosa and it calculated the mfccs, melspectrogram, chroma_stft, chroma_cq, chroma_cens and then loaded them as features

In [ ]:
sum([len(x) for x in x_data])

In [ ]:
len(data)


In [ ]:
import pickle as pkl

with open("x_data.pickle","wb") as f:
    pkl.dump(x_data, f)

with open("y_label.pickle","wb") as f:
    pkl.dump(y_label, f)

# Mount Drive

In [ ]:
# mount drive with data
from google.colab import drive
drive.mount('/content/gdrive')

# Fetch Processed Data

In [ ]:
#extracting data from pkl files
import pickle as pkl

with open("/content/gdrive/My Drive/UrbanSound8k/x_data.pickle","rb") as f:
  x_data = pkl.load(f)

with open("/content/gdrive/My Drive/UrbanSound8k/y_label.pickle","rb") as f:
  y_label = pkl.load(f)

In [ ]:
#reshape data
import numpy as np
from keras.utils.np_utils import to_categorical

X = []
Y = []
for i in range(10):
  xi = np.array(x_data[i])

  #reshaping to shape required by CNN
  xi_cnn = np.reshape(xi,(xi.shape[0],xi.shape[1],xi.shape[2], 1))
  X.append(xi_cnn)

  yi = to_categorical(y_label[i], num_classes=10)
  Y.append(yi)
  # RESHAPED THE DATA TO MAKE EACH OF THE INTO 10 CLASSES RANDOMLY AND THEN EACH CLASS HAS 873 DATAPOINTS AND ALSO CONVERTED THE Y_CLASS INTO CATEGORIES USINF TO_CATEGORICAL I.E
  # BINARY CODED CATRGORY

# Build Model

In [ ]:
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,Activation
from keras.regularizers import l2
def get_model():
  #forming model
  model = Sequential()

  #adding layers and forming the model
  model.add(Conv2D(64,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(36,5,1)))
  model.add(MaxPooling2D(padding="same"))

  model.add(Conv2D(128,kernel_size=5,strides=1,padding="same",activation="relu"))
  model.add(MaxPooling2D(padding="same"))
  model.add(Dropout(0.3))

  model.add(Flatten())

  model.add(Dense(512,activation="relu"))
  model.add(Dropout(0.4))

  model.add(Dense(512,activation="relu"))
  model.add(Dropout(0.4))

  model.add(Dense(10,activation="softmax"))

  #compiling
  model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

  return model
  # BUILT THE MODEL USING 2 CONVULATIONAL LA

# 10 Cross Validation

In [ ]:
sum_train_loss = 0
sum_test_loss = 0
sum_train_acc = 0
sum_test_acc = 0

for i in range(10):
  model = get_model()

  x_train = np.concatenate([X[j] for j in [k for k in range(10) if k not in [i]]])
  y_train = np.concatenate([Y[j] for j in [k for k in range(10) if k not in [i]]])

  x_test = X[i]
  y_test = Y[i]

  model.fit(x_train, y_train, epochs=30, batch_size=50)

  #train and test loss and scores respectively
  train_eval = model.evaluate(x_train, y_train)
  test_eval = model.evaluate(x_test, y_test)

  print("Results for folder: ", i)
  print(train_eval)
  print(test_eval)

  sum_train_loss += train_eval[0]
  sum_test_loss += test_eval[0]
  sum_train_acc += train_eval[1]
  sum_test_acc += test_eval[1]

# Result

In [ ]:
#train and test scores
print("Avg Train Loss: ", sum_train_loss/10)
print("Avg Train Acc: ", sum_train_acc/10)
print("Avg Test Loss Score: ", sum_test_loss/10)
print("Avg Test Loss Acc: ", sum_test_acc/10)

In [ ]:
Avg Train Loss:  0.10445725694298744
Avg Train Acc:  0.969599062204361
Avg Test Loss Score:  1.6439483225345612
Avg Test Loss Acc:  0.6261717498302459